In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
X_train = np.load("vectors/X_train.npy")
Y_train = np.load("vectors/Y_train_q2-4.npy")
# Y_train = Y_train.astype(int)

In [ ]:
X_valid = np.load("vectors/X_valid.npy")
Y_valid = np.load("vectors/Y_valid_q2-4.npy")
# Y_valid = Y_valid.astype(int)

In [ ]:
X_test = np.load("vectors/X_test.npy")
Y_test = np.load("vectors/Y_test_q2-4.npy")
# Y_test = Y_test.astype(int)

In [ ]:
maxlen = X_train.shape[1]
vocab_size = 26000
embeding_dimension = 10
number_of_heads = 6 

In [ ]:
norm_layer = layers.LayerNormalization(epsilon=1e-6)

In [ ]:
dropout = layers.Dropout(0.15)

In [ ]:
feednn = keras.Sequential(
    [
        layers.Dense(10, activation="relu"),
        layers.Dense(embeding_dimension),
    ]
)

In [ ]:
mha = layers.MultiHeadAttention(num_heads = number_of_heads, key_dim = embeding_dimension)

In [ ]:
token_embending = layers.Embedding(input_dim=vocab_size, output_dim=embeding_dimension)

In [ ]:
positional_embending = layers.Embedding(input_dim=maxlen, output_dim=embeding_dimension)

In [ ]:
soft_layer = tf.keras.layers.Softmax()

In [ ]:
inputs = layers.Input(shape=(maxlen,))

pos = tf.range(start=0, limit=tf.shape(inputs)[-1], delta=1)
pos = positional_embending(pos)
x = token_embending(inputs)
x += pos

a = mha(x, x)
a = dropout(a)
a = norm_layer(x + a)
f = feednn(a)
f = dropout(f)
x = layers.GlobalAveragePooling1D()(f)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(9, activation="relu")(x)
x = layers.Reshape((3, 3))(x)
outputs = soft_layer(x)

# outputs = layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile("adam", loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
# model.summary()

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size=32, epochs=15)

In [ ]:
model.evaluate(X_test, Y_test, batch_size=32)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
Y_pred = np.argmax(Y_pred, axis=2)

In [ ]:
Y_pred.shape

In [ ]:
Y_test.shape

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test[:, 0], Y_pred[:, 0])
cm_df = pd.DataFrame(cm, index=['Actual unknown', 'Actual yes', 'Actual no'], columns=['Predicted unknown', 'Predicted yes', 'Predicted no'])
cm_df

In [ ]:
cm = confusion_matrix(Y_test[:, 1], Y_pred[:, 1])
cm_df = pd.DataFrame(cm, index=['Actual unknown', 'Actual yes', 'Actual no'], columns=['Predicted unknown', 'Predicted yes', 'Predicted no'])
cm_df

In [ ]:
cm = confusion_matrix(Y_test[:, 2], Y_pred[:, 2])
cm_df = pd.DataFrame(cm, index=['Actual unknown', 'Actual yes', 'Actual no'], columns=['Predicted unknown', 'Predicted yes', 'Predicted no'])
cm_df